# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Using the data from part 1, load the data.  Since there is a incremented field, make sure to define index column
# rather than create new one.
city_data = pd.read_csv("output_data/cities.csv", index_col=0)
city_data.head()

,city,lat,lon,maxtemp,humidity,cloudiness,windspeed,country,datetime
0,east london,-33.02,27.91,72.81,79,100,11.65,ZA,1607457711
1,turukhansk,65.82,87.98,-28.86,86,52,2.33,RU,1607457763
2,hilo,19.73,-155.09,78.01,83,90,6.93,US,1607457619
3,porbandar,21.64,69.61,74.97,51,20,8.86,IN,1607457694
4,rikitea,-23.12,-134.97,76.73,78,0,16.69,PF,1607457582


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'lat' and 'lon' into  locations, humidity stored as well.
locations = city_data[["lat", "lon"]].astype(float)
humidity = city_data["humidity"]

In [5]:
# Create the heat map

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
# Set the layout to see the majority of the data points without duplication
fig = gmaps.figure(center=(0.00, 0.00), zoom_level = 2, layout=figure_layout)

# add the heat layer showing cities with their humidity as weight.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)
fig.add_layer(heat_layer)

# show the map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My ideal weather conditions are:
* Between 40 and 60 degrees
* Medium humidity - no more than 50%
* Light breeze only (5 mph or less)


In [6]:
# Create new dataframe with only the cities meeting conditions
ideal_city_data = city_data.loc[((city_data["maxtemp"] >= 40) 
                                & (city_data["maxtemp"] <=60) 
                                & (city_data["humidity"] <= 50)
                                & (city_data["windspeed"] <= 5))]
# display city data for double check
ideal_city_data

,city,lat,lon,maxtemp,humidity,cloudiness,windspeed,country,datetime
121,sakakah,29.97,40.21,59.00,41,0,3.36,SA,1607457378
259,faya,18.39,42.45,59.00,47,0,2.44,SA,1607457728
341,saint george,37.10,-113.58,57.99,22,1,3.36,US,1607457326


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a new dataframe, and add Hotel Name column
hotel_df = ideal_city_data
hotel_df["Hotel Name"] = ""

target_radius = 5000
target_type = "lodging"

for index, row in hotel_df.iterrows():
    target_coordinates = str(row['lat']) + "," + str(row['lon'])
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    hotel_df.loc[index,["Hotel Name"]] = response["results"][0]["name"]


<ipython-input-7-82fcb4329e9c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
C:\Users\julie\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
# NOTE: Do not change any of the code in this cell
# I did have to change the variable names for city and country to fit my dataframe.

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))